In [23]:
# See https://www.pyimagesearch.com/2019/07/15/video-classification-with-keras-and-deep-learning/
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import os
import pandas as pd
from tqdm import tqdm

In [19]:
def get_labels(pardir='test/data/sports/data', sort=True):
    
    from os.path import isdir, basename, join, isfile
    files = [f for f in os.listdir(pardir) if isdir(join(pardir, f))]

    train_image = []
    train_class = []

    for idir in files:
        clazz = basename(idir)
        if clazz.startswith("."):
            continue
        
        images = os.listdir(pardir+"/"+clazz)
        for i in tqdm(range(len(images))):
            
            image_name = images[i] 
            lc = image_name.lower()
            if not lc.endswith(".jpg") and not lc.endswith(".jpeg") and not lc.endswith(".png"):
                continue
            
            if isdir(join(pardir, clazz, image_name)):
                continue
                
            if lc.endswith(".ipynb_checkpoints"):
                continue
            
            train_image.append(clazz+"/"+image_name)
            train_class.append(clazz)
        
    train_data = pd.DataFrame()
    train_data['image'] = train_image
    train_data['class'] = train_class

    if sort:
        train_data = train_data.sort_values(by=['class'])
        
    return train_data

In [20]:
train = get_labels()

100%|██████████| 536/536 [00:00<00:00, 19899.68it/s]


In [24]:
def images_and_labels(pdir='test/data/sports/data', train=train):
    
    # creating an empty list
    images = []
    labels = []

    # for loop to read and store frames
    for i in tqdm(range(train.shape[0])):
        # loading the image and keeping the target size as (500,300,3)
        # The images are variable size
        impath = pdir+'/'+train['image'][i]
        if i % 1000 == 0:
            print(impath)
        try:
            image = cv2.imread(impath)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (224, 224))
            
            # appending the image to the train_image list
            images.append(image)
            labels.append(train['class'][i])
        except:
            raise Exception("Problem with {}".format(impath))
    return images, labels

In [25]:
data, labels = images_and_labels()

  0%|          | 12/14360 [00:00<04:23, 54.38it/s]

test/data/sports/data/gymnastics/00000372.jpg


  7%|▋         | 1012/14360 [00:11<02:39, 83.71it/s]

test/data/sports/data/wrestling/00000292.jpg


 14%|█▍        | 2004/14360 [00:25<03:15, 63.06it/s]

...

 91%|█████████ | 13018/14360 [02:31<00:07, 173.26it/s]

test/data/sports/data/kabaddi/00000445.jpg


 98%|█████████▊| 14030/14360 [02:36<00:01, 208.35it/s]

test/data/sports/data/shooting/00000071.jpg


100%|██████████| 14360/14360 [02:38<00:00, 90.70it/s] 


In [26]:
data = np.array(data)
labels = np.array(labels)

# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.25, stratify=labels, random_state=42)

# initialize the training data augmentation object
trainAug = ImageDataGenerator(
	rotation_range=30,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
valAug = ImageDataGenerator()

# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

In [27]:
# load the ResNet-50 network, ensuring the head FC layer sets are left
# off
baseModel = ResNet50(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))


94773248/94765736 [==============================] - 22s 0us/step


In [28]:
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(lb.classes_), activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

In [29]:
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the training process
for layer in baseModel.layers:
    layer.trainable = False

In [35]:
def compile(epochs=50, model=model) :
    # compile our model (this needs to be done after our setting our
    # layers to being non-trainable)
    print("[INFO] compiling model...")
    opt = SGD(lr=1e-4, momentum=0.9, decay=1e-4 / epochs)
    model.compile(loss="categorical_crossentropy", optimizer=opt,
        metrics=["accuracy"])

compile()


[INFO] compiling model...


In [37]:
def train(epochs=25, model=model):
    print("[INFO] training head...")
    from keras.callbacks import ModelCheckpoint
    mcp_save = ModelCheckpoint('sports_weight.h5', save_best_only=True, monitor='val_loss', mode='min')

    H = model.fit(
        x=trainAug.flow(trainX, trainY, batch_size=32),
        steps_per_epoch=len(trainX) // 32,
        validation_data=valAug.flow(testX, testY),
        validation_steps=len(testX) // 32,
        epochs=epochs,
        callbacks=[mcp_save])

train()

[INFO] training head...
Epoch 1/25
336/336 [==============================] - 48047s 143s/step - loss: 2.1506 - accuracy: 0.3833 - val_loss: 1.6808 - val_accuracy: 0.5633
Epoch 2/25
336/336 [==============================] - 397s 1s/step - loss: 1.9826 - accuracy: 0.4288 - val_loss: 1.5129 - val_accuracy: 0.6035
Epoch 3/25
336/336 [==============================] - 397s 1s/step - loss: 1.8437 - accuracy: 0.4683 - val_loss: 1.3966 - val_accuracy: 0.6250
Epoch 4/25
336/336 [==============================] - 430s 1s/step - loss: 1.7410 - accuracy: 0.4990 - val_loss: 1.2971 - val_accuracy: 0.6532
Epoch 5/25
336/336 [==============================] - 447s 1s/step - loss: 1.6566 - accuracy: 0.5232 - val_loss: 1.2279 - val_accuracy: 0.6618
Epoch 6/25
336/336 [==============================] - 418s 1s/step - loss: 1.5986 - accuracy: 0.5311 - val_loss: 1.1704 - val_accuracy: 0.6766
Epoch 7/25
336/336 [==============================] - 430s 1s/step - loss: 1.5340 - accuracy: 0.5490 - val_loss: 1

In [38]:
predictions = model.predict(x=testX.astype("float32"), batch_size=32)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=lb.classes_))

                precision    recall  f1-score   support

     badminton       0.74      0.73      0.73       232
      baseball       0.72      0.75      0.74       183
    basketball       0.69      0.70      0.70       122
        boxing       0.72      0.74      0.73       176
         chess       0.84      0.80      0.82       119
       cricket       0.79      0.73      0.76       166
       fencing       0.71      0.77      0.74       156
      football       0.76      0.88      0.81       196
      formula1       0.84      0.84      0.84       169
    gymnastics       0.83      0.67      0.74       178
        hockey       0.76      0.55      0.64       142
    ice_hockey       0.85      0.94      0.89       177
       kabaddi       0.63      0.80      0.70       113
        motogp       0.97      0.84      0.90       167
      shooting       0.87      0.77      0.82       133
      swimming       0.94      0.95      0.95       171
  table_tennis       0.80      0.74      0.77  

In [39]:
model.save("sports_model.h5", save_format="h5")

# serialize the label binarizer to disk
f = open("sports_labels.pkl", "wb")
f.write(pickle.dumps(lb))
f.close()